In [4]:
!pip install pylidc
!pip install pydicom
!pip install opencv-python

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [5]:
import numpy as np
import pylidc as pl
import pydicom
import cv2
from statistics import median_high
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, BatchNormalization, Activation, AveragePooling2D, concatenate
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint, EarlyStopping

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, classification_report, recall_score, f1_score


In [6]:
import os
import pydicom
import pylidc as pl
import numpy as np
import cv2
from pylidc.Contour import Contour
from statistics import median_high

# Paths to the directories containing DICOM files
base_dirs = [
    "/kaggle/input/lidcidri30/LIDC-IDRI-0001-0200",
    "/kaggle/input/lidcidri30/LIDC-IDRI-0201-0400",
    "/kaggle/input/lidcidri30/LIDC-IDRI-0401-0600"
]

images = []
labels = []
malignant_count = 0
benign_count = 0
max_malignant_count = 4500  # Target number of malignant images
max_benign_count = 2500     # Target number of benign images

# Patch the Contour class method to_matrix
def patched_to_matrix(self, include_k=True):
    coords = np.array([list(map(int, coord.split(','))) for coord in self.coords.split('\n')])
    ij = np.c_[coords[:, 0], coords[:, 1]]

    if not include_k:
        return ij.astype(np.int64)

    k = np.ones(ij.shape[0]) * self.image_z_position
    return np.c_[ij, k].astype(np.int64)

Contour.to_matrix = patched_to_matrix

def calculate_malignancy(nodule):
    list_of_malignancy = [annotation.malignancy for annotation in nodule]
    malignancy = median_high(list_of_malignancy)
    return 'malignant' if malignancy > 3 else 'benign'

# Process DICOM files with the patched Contour class
for base_dir in base_dirs:
    for dirpath, _, filenames in os.walk(base_dir):
        for filename in filenames:
            if filename.endswith(".dcm"):
                try:
                    dicom_file = os.path.join(dirpath, filename)
                    dicom_data = pydicom.dcmread(dicom_file)
                    pid = dicom_data.PatientID
                    scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == pid).first()

                    if scan is None:
                        continue

                    nodules_annotation = scan.cluster_annotations()

                    try:
                        if len(nodules_annotation) > 0:
                            contains_malignant = False

                            for nodule in nodules_annotation:
                                malignancy = calculate_malignancy(nodule)
                                if malignancy == 'malignant':
                                    contains_malignant = True
                                    break

                            if contains_malignant:
                                if malignant_count >= max_malignant_count:
                                    continue
                                malignancy = 'malignant'
                                malignant_count += 1
                            else:
                                if benign_count >= max_benign_count:
                                    continue
                                malignancy = 'benign'
                                benign_count += 1

                            img = dicom_data.pixel_array  # Assuming DICOM contains pixel data directly
                            img = cv2.resize(img, (300, 300))  # Resize to 300x300 for DensNet201
                            img = img.astype(np.float32)

                            images.append(img)
                            labels.append(malignancy)

                            print(f"Processed {malignant_count + benign_count} files")
                            print(f"Malignancy: {malignancy}")
                            print(f"Patient ID: {pid}")

                        if malignant_count >= max_malignant_count and benign_count >= max_benign_count:
                            break
                    except Exception as e:
                        # Skip problematic nodules that cannot be grouped manually
                        print(f"Skipping nodule processing for Patient ID {pid} due to error: {e}")

                except Exception as e:
                    print(f"Error processing file {filename}: {e}")

        if malignant_count >= max_malignant_count and benign_count >= max_benign_count:
            break

print(f"Total malignant images: {malignant_count}")
print(f"Total benign images: {benign_count}")


Processed 1 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 2 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 3 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 4 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 5 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 6 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 7 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 8 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 9 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 10 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 11 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 12 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 13 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 14 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 15 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 16 files
Malignancy: ben

In [7]:
# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

In [8]:
# Normalization
images = images/255.0

In [9]:
# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_encoded = to_categorical(labels_encoded)

# Print the mapping of labels to their encoded values
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Mapping:", label_mapping)


Label Mapping: {'benign': 0, 'malignant': 1}


In [10]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(images, labels_encoded, test_size=0.2, random_state=42, shuffle=True)

# Convert to numpy arrays and reshape
X_train = np.array(X_train).reshape(-1, 300, 300, 1)
X_test = np.array(X_test).reshape(-1, 300, 300, 1)

# Define number of classes
num_classes = len(np.unique(labels_encoded))

print(num_classes)

2


In [11]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model


# Load the EfficientNet B7 model with random weights
base_model = EfficientNetB2(weights=None, include_top=False, input_shape=(300, 300, 1))

# Add a global average pooling layer
x = GlobalAveragePooling2D()(base_model.output)

# Add a fully connected dense layer with 1024 units and ReLU activation
x = Dense(1024, activation='relu')(x)

# Add a dropout layer for regularization
x = tf.keras.layers.Dropout(0.5)(x)

# Add the output layer for 2 classes
output = Dense(2, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=output)


In [12]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 300, 300,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 300, 300,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 300, 300,  │          3 │ rescaling[0][0]   │
│ (Normalization)     │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 301, 301,  │          0 │ normalization[0]… │
│ (ZeroPadding2D)     │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 150, 150,  │        288 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 150, 150,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 150, 150,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 150, 150,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 150, 150,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 150, 150,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 150, 150,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 150, 150,  │        512 │ block1a_se_excit… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_bn  │ (None, 150, 150,  │         64 │ block1a_project_

 Total params: 9,212,855 (35.14 MB)

 Trainable params: 9,145,284 (34.89 MB)

 Non-trainable params: 67,571 (263.95 KB)

In [14]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model



k = 5  # Number of folds for cross-validation
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Placeholder arrays to store performance metrics
accuracy_scores = []

# Perform K-Fold Cross-Validation
for fold, (train_index, val_index) in enumerate(kf.split(X_train)):
    print(f'Fold {fold + 1}/{k}')
    
    # Split data into training and validation sets for this fold
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
    
    # Reinitialize base model and output layers for each fold
    base_model = EfficientNetB2(weights=None, include_top=False, input_shape=(300, 300, 1))
    
    # Add layers to the model
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(2, activation='softmax')(x)  # 2 output classes for binary classification
    
    # Create the model
    model = Model(inputs=base_model.input, outputs=output)
    
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    # Define checkpoint filepath
    checkpoint_filepath = f'./best_model_fold_{fold + 1}.keras'
    
    # Define callbacks for early stopping and model checkpointing
    checkpoint = ModelCheckpoint(checkpoint_filepath, 
                                 monitor='val_accuracy', 
                                 save_best_only=True, 
                                 mode='max', 
                                 verbose=1)

    early_stopping = EarlyStopping(monitor='val_accuracy', 
                                   patience=4,  # Stop training if no improvement after 4 epochs
                                   mode='max',
                                   verbose=1,
                                   restore_best_weights=True)
    
    # Train the model with early stopping and checkpoint callbacks
    model.fit(X_train_fold, y_train_fold, 
              epochs=5,  # Increase epochs for better learning
              batch_size=16, 
              validation_data=(X_val_fold, y_val_fold),
              verbose=1,
              callbacks=[checkpoint, early_stopping])

    # Load the best model saved during this fold
    model = load_model(checkpoint_filepath)

    # Evaluate the model on the validation fold
    scores = model.evaluate(X_val_fold, y_val_fold, verbose=0)
    accuracy_scores.append(scores[1])  # Save accuracy from the evaluation

# Calculate and print average accuracy across all folds
avg_accuracy = np.mean(accuracy_scores)
print(f'Average Accuracy across {k}-folds: {avg_accuracy:.4f}')


Fold 1/5
Epoch 1/5
280/280 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step - accuracy: 0.5927 - loss: 0.8628
Epoch 1: val_accuracy improved from -inf to 0.34107, saving model to ./best_model_fold_1.keras
280/280 ━━━━━━━━━━━━━━━━━━━━ 195s 221ms/step - accuracy: 0.5930 - loss: 0.8621 - val_accuracy: 0.3411 - val_loss: 1.2544
Epoch 2/5
280/280 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step - accuracy: 0.9441 - loss: 0.1572
Epoch 2: val_accuracy improved from 0.34107 to 0.45268, saving model to ./best_model_fold_1.keras
280/280 ━━━━━━━━━━━━━━━━━━━━ 54s 192ms/step - accuracy: 0.9442 - loss: 0.1571 - val_accuracy: 0.4527 - val_loss: 1.2454
Epoch 3/5
280/280 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step - accuracy: 0.9848 - loss: 0.0511
Epoch 3: val_accuracy improved from 0.45268 to 0.99107, saving model to ./best_model_fold_1.keras
280/280 ━━━━━━━━━━━━━━━━━━━━ 54s 191ms/step - accuracy: 0.9848 - loss: 0.0511 - val_accuracy: 0.9911 - val_loss: 0.0210
Epoch 4/5
280/280 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step - accuracy: 0.9884 - loss